In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
import datetime
from sklearn.preprocessing import LabelEncoder

In [1]:
pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 7.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 15.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 7.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
rev_df = pd.read_csv('datasets/User_Orders.csv')

rev_df.head()

,orderNumber,orderedDate,customerID,customerName,asin
0,93f35a6b9,"09 22, 2013",A1BB77SEBQT8VX,Darrow H Ankrum II,['B00007GDFV']
1,d6ffc1262,"07 17, 2013",AHWOW7D1ABO9C,rosieO,['B00007GDFV']
2,b52b6a408,"04 13, 2013",AKS3GULZE0HFC,M. Waltman,['B00007GDFV']
3,d6eba872b,"03 9, 2013",A38NS6NF6WPXS,BTDoxies,['B00007GDFV']
4,87e6fdc25,"01 27, 2013",A1KOKO3HTSAI1H,Robin Howard,['B00007GDFV']


In [3]:
# Get the number of rows using the shape attribute
num_rows = rev_df.shape[0]
print("Number of rows:", num_rows)

Number of rows: 180109


In [4]:
# Calculate the number of rows for the first half
half_rows = num_rows // 2

# Create a subset of the DataFrame containing the first half
first_half_df = rev_df.iloc[:half_rows]

# Get the number of rows in the first half subset
num_rows_first_half = first_half_df.shape[0]
print("Number of rows in the first half subset:", num_rows_first_half)

Number of rows in the first half subset: 90054


In [5]:
rev_df = rev_df.drop(['orderedDate', 'customerID', 'customerName'], axis=1)
rev_df

,orderNumber,asin
0,93f35a6b9,['B00007GDFV']
1,d6ffc1262,['B00007GDFV']
2,b52b6a408,['B00007GDFV']
3,d6eba872b,['B00007GDFV']
4,87e6fdc25,['B00007GDFV']
...,...,...
180104,afad3a759,['B01HIX20DQ']
180105,7202904a2,['B01HIX20DQ']
180106,6e2adc762,['B01HIYPFXC']
180107,a9f56573f,['B01HJ6HHD0']


In [6]:
rev_df['asin'] = rev_df['asin'].str.replace("'", "")
rev_df.iloc[60:71]

,orderNumber,asin
60,01c7ae5db,[B00007GDFV]
61,f737aff98,[B00007GDFV]
62,8e5c05feb,[B00007GDFV]
63,f7d950557,"[B00007GDFV, B00B6AWXYE, B00EZMCOCQ]"
64,d6b4228ed,[B00007GDFV]
65,eca3d5363,[B00007GDFV]
66,2faa6025b,[B00007GDFV]
67,bd7540f73,[B00007GDFV]
68,2cec1b58b,[B00007GDFV]
69,38914494c,[B00007GDFV]


In [7]:
# Remove square brackets and split the string
rev_df['asin'] = rev_df['asin'].str.strip('[]').str.split(', ')

# Explode the column
result = rev_df.explode('asin')
result.iloc[60:71]

,orderNumber,asin
60,01c7ae5db,B00007GDFV
61,f737aff98,B00007GDFV
62,8e5c05feb,B00007GDFV
63,f7d950557,B00007GDFV
63,f7d950557,B00B6AWXYE
63,f7d950557,B00EZMCOCQ
64,d6b4228ed,B00007GDFV
65,eca3d5363,B00007GDFV
66,2faa6025b,B00007GDFV
67,bd7540f73,B00007GDFV


In [8]:
first_half_df = first_half_df.drop(['orderedDate', 'customerID', 'customerName'], axis=1)
first_half_df

,orderNumber,asin
0,93f35a6b9,['B00007GDFV']
1,d6ffc1262,['B00007GDFV']
2,b52b6a408,['B00007GDFV']
3,d6eba872b,['B00007GDFV']
4,87e6fdc25,['B00007GDFV']
...,...,...
90049,596ff698b,['B00WQY61CO']
90050,a78101146,['B00WQY61CO']
90051,1731f6eac,['B00WQY61CO']
90052,15dc6016f,['B00WQY61CO']


In [9]:
first_half_df['asin'] = first_half_df['asin'].str.replace("'", "")
first_half_df.iloc[60:71]

,orderNumber,asin
60,01c7ae5db,[B00007GDFV]
61,f737aff98,[B00007GDFV]
62,8e5c05feb,[B00007GDFV]
63,f7d950557,"[B00007GDFV, B00B6AWXYE, B00EZMCOCQ]"
64,d6b4228ed,[B00007GDFV]
65,eca3d5363,[B00007GDFV]
66,2faa6025b,[B00007GDFV]
67,bd7540f73,[B00007GDFV]
68,2cec1b58b,[B00007GDFV]
69,38914494c,[B00007GDFV]


In [10]:
# Remove square brackets and split the string
first_half_df['asin'] = first_half_df['asin'].str.strip('[]').str.split(', ')

# Explode the column
first_half_result = first_half_df.explode('asin')
first_half_result.iloc[60:71]

,orderNumber,asin
60,01c7ae5db,B00007GDFV
61,f737aff98,B00007GDFV
62,8e5c05feb,B00007GDFV
63,f7d950557,B00007GDFV
63,f7d950557,B00B6AWXYE
63,f7d950557,B00EZMCOCQ
64,d6b4228ed,B00007GDFV
65,eca3d5363,B00007GDFV
66,2faa6025b,B00007GDFV
67,bd7540f73,B00007GDFV


In [11]:
pivot_df = first_half_result.pivot_table(index='orderNumber', columns='asin', aggfunc='size', fill_value=0)


In [12]:
# Calculate cosine similarity
cosine_sim = cosine_similarity(pivot_df)

In [13]:
import numpy as np
import joblib

# Save the cosine similarity matrix to a file
cosine_sim_filename = 'cosine_similarity_matrix.npy'
np.save(cosine_sim_filename, cosine_sim)
print(f"Cosine similarity matrix saved as {cosine_sim_filename}")

OSError: Not enough free space to write 64877783328 bytes

In [14]:
# Convert ASINs to numerical labels
le = LabelEncoder()
asin_labels = le.fit_transform(result['asin'])
asin_mapping = dict(zip(le.classes_, asin_labels))

In [15]:
# Save the LabelEncoder and ASIN mapping
le_filename = 'label_encoder.joblib'
joblib.dump(le, le_filename)
print(f"LabelEncoder saved as {le_filename}")

asin_mapping_filename = 'asin_mapping.joblib'
joblib.dump(asin_mapping, asin_mapping_filename)
print(f"ASIN mapping saved as {asin_mapping_filename}")

LabelEncoder saved as label_encoder.joblib
ASIN mapping saved as asin_mapping.joblib


In [14]:
# Function to get product recommendations
def get_recommendations(asin, num_recommendations=3):
    asin_label = asin_mapping.get(asin)
    if asin_label is None:
        return "ASIN not found in the dataset"

    # Calculate similarity scores for the given ASIN
    sim_scores = list(enumerate(cosine_sim[asin_label]))

    # Sort the products based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get top N similar products
    top_products = sim_scores[1:num_recommendations+1]
    top_product_labels = [item[0] for item in top_products]
    top_product_asins = le.inverse_transform(top_product_labels)

    return top_product_asins

In [15]:
# Example usage
input_asin = 'B00B6AWXYE'
recommendations = get_recommendations(input_asin)
print(f"Top 3 recommendations for {input_asin}: {recommendations}")

Top 3 recommendations for B00B6AWXYE: ['B0006NW5O0' 'B000GHRZN2' 'B000H34CL8']
